# RAG - 1. indexing(OpenAI)

> 1) indexing 작업 (문서를 읽어서, vectorstore에 저장하기)
> 2) 목적 : langchain기능 사용해보기, web으로 문서파일을 업로드하고, document을 읽어서, split해서, chroma에 추가하는 기능
> 3) 확장해보기 : text, pdf등 문서읽기로 확장
> 4) 선행작업 : OPENAI_API_KEY 등록후 획득


In [9]:
# 1) OPENAI_API_KEY를 먼저 확보하고, ".env"파일에 저장해두어야 한다.


import os
from dotenv import load_dotenv

load_dotenv("/home/mhkwon/.env")
openai_api_key = os.getenv("OPENAI_API_KEY")
#print(openai_api_key)

In [10]:
# 2) vdb 저장 경로 및 읽어올 문서의 url

VECTORSTORE_PATH = 'chroma_store'

DOCUMENT_URL = 'https://www.etnews.com/20240823000244'

In [11]:
from langchain_openai import OpenAIEmbeddings


embeddings = OpenAIEmbeddings()

In [12]:
# 3) 문서를 읽어서, 분리해서, 저장하기

from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter

import json
import re

vectorstore = Chroma(persist_directory=VECTORSTORE_PATH, embedding_function = embeddings )


#### 웹사이트에서 정보를 읽어들이는 함수

def load_docs(url=DOCUMENT_URL):
    loader = WebBaseLoader(web_paths=(url, ), )
    docs = loader.load()
    
    #print(docs)
    return docs


#### 문서를 특정 길이(dkfosms 1000바이트)로 쪼개(chunk)는 작업
def split(docs):
    # Restructure to process the info in chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(docs)
    
    #print(splits)
    cleaned = []
    for doc in splits:
        #doc.page_content.replace("\n","")
        doc.page_content = re.sub(r"[\r\n\\]","", doc.page_content)
        doc.page_content = re.sub(r"\t+"," ", doc.page_content)
        cleaned.append(doc)

    return cleaned


#### 벡터스토어에 저장하기
def store(splits):
    vectorstore.add_documents(documents=splits)


#### 메인
def loadstore(url):
    #print('****** url=', url)

    docs = load_docs(url)
    splits = split(docs)
    store(splits)

    response_json = {
        #"docs": docs,
        "splits" : splits,
    }
    my_json = json.dumps(response_json, default=str, ensure_ascii=False, indent=8, separators =(",\n", " = ") )

    return my_json



In [13]:
# 웹을 읽어서, vdb에 저장한 내용 확인하기


response = loadstore(DOCUMENT_URL)
response


'{\n        "splits" = [\n                "page_content=\'카카오브레인, AI 헬스케어 전담 \'씨엑스알랩\'으로 분할한다 - 전자신문   로그인회원가입SWIT경제전자모빌리티플랫폼/유통과학정치오피니언뉴스속보SWIT경제전자모빌리티플랫폼/유통과학\xa0정치오피니언국제전국스포츠특집연재라이프연예포토공연전시생활문화비주얼IT이슈플러스Hot 영상뷰포인트인포그래픽부가서비스ConferenceallshowTV시사용어PDF서비스서비스안내신문구독신청콘텐츠구매초판서비스회원서비스내 스크랩이용안내지면광고안내행사문의디지털광고안내이용약관개인정보취급방침고충처리회사소개전자신문전자신문인터넷연혁CI소개회사위치 플랫폼/유통 플랫폼 카카오브레인, AI 헬스케어 전담 \'씨엑스알랩\'으로 분할한다발행일 : 2024-08-23 17:05공유하기페이스북X(트위터)메일URL 복사글자크기 설정가작게가보통가크게〈자료 카카오브레인〉카카오브레인이 인공지능(AI) 기반 헬스케어 전문회사 \'씨엑스알랩\'으로 분할한다.카카오브레인은 오는 10월 1일 카카오의 정보기술(IT) 개발 자회사 디케이테크인에 흡수합병된 후 별도 법인 \'씨엑스알랩\'으로 물적 분할한다고 23일 공시했다.카카오브레인은 지난 5월 대부분의 사업부문이 카카오 본사에 흡수합병된 후 흉부 엑스레이 판독 보조 AI 기술 중심의 헬스케어 조직만 남아있다. 오는 10월 씨엑스알랩으로 분할된 후 독자적으로 AI 기반 헬스케어 사업을 추진한다.\' metadata={\'source\': \'https://www.etnews.com/20240823000244\', \'title\': \\"카카오브레인, AI 헬스케어 전담 \'씨엑스알랩\'으로 분할한다 - 전자신문\\", \'description\': \'미래를 보는 창 - 전자신문\', \'language\': \'ko\'}",\n\n                "page_content=\'카카오브레인은 지난 5월 대부분의 사업부문이 카카오 본사에 흡수합병된 후 흉부


# 2단계 - 검색(query) 단계



In [14]:

from langchain_chroma import Chroma

#저장된 vectorstore을 읽어들인다.

vectordb = Chroma(persist_directory=VECTORSTORE_PATH, embedding_function=embeddings)

In [21]:
# 다음 문장과 유사한 문장을 저장된 vdb에서 찾아낸다.

prompt = '미래형 가상화와 디바이스 관리는 무엇인가?'

search = vectordb.similarity_search_with_score(prompt)

In [22]:
# 아래 결과는 저장된 문서에 가장 유사한 문장을 찾아 낸것이다.
# 유사성정도의 수치는 마지막 숫자로, 1.0에 가까울수록 높다.

search

[(Document(metadata={'description': '미래를 보는 창 - 전자신문', 'language': 'ko', 'source': 'https://www.etnews.com/20240823000244', 'title': "카카오브레인, AI 헬스케어 전담 '씨엑스알랩'으로 분할한다 - 전자신문"}, page_content='주요 행사 ❯나만의 AI 전문가 고용하기:Claude, GPTs, API 활용 Agentic Workflow 구축 1-day 워크숍K-클라우드 네이티브 서밋 2024디지털 전환 시대의 새로운 보안 패러다임: SASE 기술 트렌드와 제로 트러스트 업무 환경 구현2024 시큐어 스페이스 서밋AI에이전트가 가져올비즈니스 변화와 기업의 대응생성형 AI로 한단계 진화하는 AICC : 고객 경험과 생산성 향상을 동시에진화하는 ERP, Copilot과 함께하는 Dynamics 365 Finance & OperationsChatGPT(GPT4o)와 Tableau를 활용한‘프로 마케터’ 되기 1-day 워크숍BIZPLUS[올쇼TV] 생성형 AI로 한 단계 진화하는 AICC-고객 경험과 생산성 향상을 동시에 12월 5일 생방송"SASE 기술 트렌드와 제로 트러스트 업무 환경 구현” 11월 27일 생방송[올쇼TV] Horizon과 Workspace ONE으로 구축하는 미래형 가상화와 디바이스 관리 11월 27일 생방송“AI 에이전트가 가져올 비즈니스 변화와 기업의 대응” 세미나 12월 5일 개최HOT 영상 ❯현역 피하려고 105kg까지 불린 20대 남성햄스터 때문에 비행기가 4일 동안 멈춘 사연AI 로봇 투입한 BMW…작업속도 \'4배\' 증가차 부수고 시민 위협한 20대 음주 운전자뉴스속보SWIT경제전자모빌리티플랫폼/유통과학\xa0정치오피니언국제전국스포츠특집연재라이프연예포토공연전시생활문화비주얼IT이슈플러스Hot 영상뷰포인트인포그래픽'),
  0.35520342525834936),
 (Document(metadata={'description'